In [ ]:
# install langchain and openai

In [8]:
from langchain import OpenAI
import os
key = os.environ.get('OPENAI_API_KEY')
llm = OpenAI(
    openai_api_key = key,
    temperature=0,
    model_name="text-davinci-003"
)

In [2]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

/Users/johnwilliams/Library/Python/3.9/lib/python/site-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [3]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [4]:
# this achieves the same thing as above but with less code
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)



In [5]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [6]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [7]:
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
 I need to calculate this expression
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought: I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

Check the answer

In [11]:

(4.5*2.1)**2.2

139.94261298333066

In [12]:
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")



> Entering new AgentExecutor chain...
 I need to figure out how many apples are in the boxes
Action: Calculator
Action Input: 8 * 2.5
Observation: Answer: 20.0
Thought: I need to add the apples Mary has to the apples in the boxes
Action: Calculator
Action Input: 4 + 20.0
Observation: Answer: 24.0
Thought: I now know the final answer
Final Answer: We have 24 apples.

> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': 'We have 24 apples.'}

In [14]:

zero_shot_agent("what is the capital of Norway?")

In [15]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)

In [17]:
tools.append(llm_tool)

# reinitialize the agent
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [18]:
zero_shot_agent("what is the capital of Norway?")



> Entering new AgentExecutor chain...
 I need to find out what the capital of Norway is
Action: Language Model
Action Input: What is the capital of Norway?
Observation: 

The capital of Norway is Oslo.
Thought: I now know the final answer
Final Answer: The capital of Norway is Oslo.

> Finished chain.


{'input': 'what is the capital of Norway?',
 'output': 'The capital of Norway is Oslo.'}